In [1]:
from pymeasure.instruments.lighthousephotonics import Sprout #import laser codes
laser_power = Sprout('COM4')
laser_power.adapter.connection.baud_rate = 19200
laser_power.adapter.connection.read_termination = '\r'
laser_power.adapter.connection.write_termination = '\r'
laser_power.power = 0.01
import time
from datetime import timedelta
import rpy2.robjects as robjects

from newportxps import NewportXPS #motion controller
from threading import Thread
# from thread import MotionThread, MirrorThread
import os, sys

import numpy as np
import pandas as pd
from write_utils import write_data_files, get_mean, get_move_y, duplicate_to_dataset

from flipper import mirror #import flipper mirror codes
from spectra import capture_photo #import spectrometer codes

from pathlib import Path


c:\Users\UWAdmin\Anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)
c:\Users\UWAdmin\Anaconda3\lib\site-packages\pyvisa\highlevel.py:3355: FutureWarning: get_instrument is deprecated and will be removed in 1.12, use open_resource instead.
  FutureWarning,


In [2]:
path = Path(r'c:\\Users\\UWAdmin\\Desktop\\_pyControl\\campaigns')
campaign_path = path / "2023-03-10-BATCH-1"
# os.mkdir(campaign_path)
os.chdir(campaign_path)

In [4]:
write_data_files(path, series=1, nr_random_lines=8, \
    power_range=(100,2600), time_range=(2000, 14000), pressure_range=(60,800), batch=True)

TODAY's DATE: 2023-03-10
POWER: (100, 2600)
 TIME: (2000, 14000)
 PRESSURE: (60, 800)
Here is the new Campaign Folder:2023-03-10-BATCH-1


In [3]:
XPSD_REMOTE_IP = '192.168.254.254'

xps = NewportXPS(XPSD_REMOTE_IP)
print(xps.status_report())

#pos_all() to get position of all axes
def pos_all():
    for sname, _ in xps.stages.items():
        print('{}: {}'.format(sname, xps.get_stage_position(sname)))
        
xps.kill_group('XYZ')
xps.initialize_allgroups()
xps.home_allgroups()

# XPS host:         192.168.254.254 (XPS-1B81)
# Firmware:         XPS-D-N13019
# Current Time:     Wed Mar 22 17:48:44 2023
# Last Reboot:      Wed Mar 22 14:06:01 2023
# Trajectory Group: XYZ
# Groups and Stages
XYZ (multipleaxesinuse), Status: Ready state from motion
   XYZ.X (MFA@MFA-CC@XPS-DRV11)
      Hardware Status: First driver powered on - ZM low level
      Positioner Errors: OK
   XYZ.Y (MFA@MFA-CC@XPS-DRV11)
      Hardware Status: First driver powered on - ZM low level
      Positioner Errors: OK
   XYZ.Z (MFA@MFA-CC@XPS-DRV11_Z)
      Hardware Status: First driver powered on - ZM low level
      Positioner Errors: OK


In [4]:
capture_photo("start",2,1,0) 


In [5]:
from pressure import close_valve,open_valve,close_all,current_pressure,gopr, to_ambient, to_vacuum, quick_fill
current_pressure()
# gopr(500)

c:\Users\UWAdmin\Anaconda3\lib\site-packages\pyvisa\highlevel.py:3355: FutureWarning: get_instrument is deprecated and will be removed in 1.12, use open_resource instead.
  FutureWarning,


121

# (E) To make sure that the material gets laser beam of max. intensity , focusing of z axis needs to be properly adjusted. 

In [6]:
z=3.49
xps.move_stage('XYZ.Z', z)
xps.move_stage('XYZ.X', 4.5)
xps.move_stage('XYZ.Y', 12)
pos_all()

XYZ.X: 4.5000016524
XYZ.Y: 11.9996976168
XYZ.Z: 3.4900030908


In [7]:

class MotionThread(Thread):
   
    def run(self):
        import pandas as pd
        d = pd.read_csv('foreward.trj',header=None) # scantime 2 (see segment 1 or row1)
        d = d.loc[:, (d != 0).any(axis=0)] # remove last 3 cols with 0
        d.columns = ['ramptime','rampdist','rampvel']
        total_time=np.sum(d['ramptime'])
        
        if d['ramptime'][0]<0.5:
            time.sleep(.5-d['ramptime'][0])
        print("MOTION THREAD\n")
        print("XPS run trajectory")
        xps.run_trajectory('foreward',)
        print('Total time {} in class'.format(total_time))
        time.sleep(total_time)
        print("TOTAL_TIME DONE, LASER 0.5W")
        laser_power.power=0.1
        print("finished and current position is:\n")
        pos_all()

class MirrorThread(Thread):

    def run(self):
        import pandas as pd
        d = pd.read_csv('foreward.trj',header=None) # scantime 2 (see segment 1 or row1)
        d = d.loc[:, (d != 0).any(axis=0)] # remove last 3 cols with 0
        d.columns = ['ramptime','rampdist','rampvel']
        mirror_sleep=d['ramptime'][0]-.5
        print("MIRROR THREAD")
        for i in range(2):
            if i==0:
                print(d['ramptime'][0])
                if d['ramptime'][0]<0.5:
                    print("mirror on")
       
                    start_time = time.monotonic()
                    mirror('on')
       
                    end_time = time.monotonic()
                    print(timedelta(seconds=end_time - start_time))
                    time.sleep(d['ramptime'][1]) #time for linear line    

                else:
                    print('{} > 0.5'.format(d['ramptime'][0]))
                    print('Mirror sleep {} in class'.format(mirror_sleep))
                    time.sleep(mirror_sleep)
                
                    print("mirror on")
                    start_time = time.monotonic()
                    mirror('on')
                    end_time = time.monotonic()
                    print(timedelta(seconds=end_time - start_time))
                    time.sleep(d['ramptime'][1]) #time for linear line
            else:
                mirror('off')
                print("mirror off")

# (H) Define coordinate details

**Takes input for**  
  
-number of lines

-starting position of x

-starting position of y

-vertical gap/step size

# (I) After measuring all resistance in dataset. MAKE A COPY OF data.csv. Get resistance mean from the last 2 columns

# (J) Define MBO

In [8]:
robjects.r('''
  experiment<-function(s){

  presToR <- function (x) {(x - 50) / 10}
  presToExp <- function (x) {x * 10 + 50}

  if(s==1){ #BO called first time to initialize and propose first point
  require(XML)

    presToR <- function (x) {(x - 50) / 10}
    presToExp <- function (x) {x * 10 + 50}

    suppressWarnings({suppressMessages({library(mlrMBO)
                                        library(ggplot2)
                                        library(doMC)
                                      })})

    ps = makeParamSet(
      makeIntegerParam("power", lower = 100, upper = 2600),
      makeIntegerParam("time", lower = 2000, upper = 14000),
      makeIntegerParam("pressure", lower = 1, upper = 75, trafo = function(x) x * 10 + 50)
    )

    ctrl = makeMBOControl(y.name = "resistance", propose.points = 8L)
    ctrl = setMBOControlInfill(ctrl, opt = "focussearch", 
                              opt.focussearch.maxit = 10,
                              opt.focussearch.points = 10000,
                              crit = makeMBOInfillCritEI())

    ctrl = setMBOControlMultiPoint(ctrl, method = "cl", cl.lie = min)
    ctrl = setMBOControlTermination(ctrl, iters = 1)

    data = read.csv("data.csv", colClasses = c(NA, NA, NA, "NULL", NA))
    data$pressure <- presToR(data$pressure)

    suppressMessages({opt.state = initSMBO(par.set = ps, 
                                          design = data,
                                          control = ctrl, 
                                          minimize = TRUE, 
                                          noisy = TRUE)})
    print("Proposed parameters:")
    prop = suppressWarnings({proposePoints(opt.state)})
    print(prop$prop.points)
    print(prop$crit.components)
    saveRDS(prop,'./prop.rds')
    saveRDS(opt.state,'./opt.state.rds')
    dummy_ratio = 0
    dummy_resistance = 0
    cat("Expected value (upper bound):\n")
    cat(paste(prop$crit.components$mean, " (",
              prop$crit.components$mean + prop$crit.components$se, ")\n", sep = ""))

    prop$prop.points$pressure <- presToExp(prop$prop.points$pressure)

    x <- data.frame(prop$prop.points, dummy_ratio, dummy_resistance,
                    prop$crit.components$mean,
                    prop$crit.components$mean + prop$crit.components$se, 
                    2*prop$crit.components$mean + prop$crit.components$se)

    x2 <- c(prop$prop.points)

    write.table(x2, file = "data.csv", sep = ",",
                append = TRUE, quote = FALSE,col.names = FALSE, row.names = FALSE)

    write.table(x, file = "plot_data.csv", sep = ",",
                append = TRUE, quote = FALSE,col.names = FALSE, row.names = FALSE)

  }
  else{
    
    presToR <- function (x) {(x-50)/10}
    presToExp <- function (x) {x*10+50}

    prop=readRDS('./prop.rds')
    opt.state=readRDS('./opt.state.rds')
    library(mlrMBO)
    library(R.utils)
    print(prop$crit.components)
    print(prop$prop.points)

    print("Proposed parameters:")
    l2keep <- 8
    nL <- countLines("data.csv")
    s <- read.csv("data.csv", header=FALSE, skip=nL-l2keep)['V5']
    updateSMBO(opt.state, x = prop$prop.points, y = as.list(s$V5))
    prop <- suppressWarnings({proposePoints(opt.state)})
    print(prop$prop.points)
    print(prop$crit.components)
    saveRDS(prop,'./prop.rds')
    saveRDS(opt.state,'./opt.state.rds')

    dummy_ratio = 0
    dummy_resistance = 0
    cat("Expected value (upper bound):\n")
    cat(paste(prop$crit.components$mean, " (",
              prop$crit.components$mean + prop$crit.components$se, ")\n", sep = ""))

    prop$prop.points$pressure <- presToExp(prop$prop.points$pressure)

    x <- data.frame(prop$prop.points, dummy_ratio, dummy_resistance,
                    prop$crit.components$mean,
                    prop$crit.components$mean + prop$crit.components$se, 
                    prop$crit.components$mean + prop$crit.components$mean + prop$crit.components$se)
    x2<-c(prop$prop.points)

    write.table(x2, file = "data.csv", sep = ",",
                append = TRUE, quote = FALSE,col.names = FALSE, row.names = FALSE)

    write.table(x, file = "plot_data.csv", sep = ",",
                append = TRUE, quote = FALSE,col.names = FALSE, row.names = FALSE)

  }}

  ''')

<rpy2.robjects.functions.SignatureTranslatedFunction object at 0x00000253DE26E388> [RTYPES.CLOSXP]
R classes: ('function',)

# (L) Propose next 12 samples

In [9]:
# campaign_path = path / "2022-04-26-BATCH-2"
# os.chdir(campaign_path)
import pandas as pd

def _printing_line_count(data:pd.DataFrame, measured_value:str = 'resistance'):
    printed_lines_count = len(data['resistance'].dropna(inplace=False))
    return printed_lines_count

def _first_line_to_print(col:int=0, row:int=0):
    None
    
def _lines_to_print(data:pd.DataFrame, line_count:int):
    printed_count = _printing_line_count(data)
    print_line_set = data.loc[printed_count::]
    return print_line_set

def get_line_numbers(data:pd.DataFrame, printing_line_count:int, measured_value:str='resistance'):
    printed_lines = _printing_line_count(data, measured_value)
    line_numbers = list(range(printed_lines, printed_lines+printing_line_count))
    return line_numbers

def get_line_parameters(data:pd.DataFrame, printing_line_count:int):
    parameters:pd.DataFrame = data.loc[get_line_numbers(data,printing_line_count)]
    return parameters.reset_index()

def generate_line_coordinates(line_count:int, y_start_location:float=6.5, x_start_location:list = [1.5, 4.5]):
    lines_per_column = int(line_count/2)
    y_stop_location: float = 11
    y_coordinates = np.linspace(y_start_location, y_stop_location, lines_per_column)
    coordinates = []
    for x in x_start_location:
        for y in y_coordinates:
            coordinates.append((x,y))

    return coordinates

data = pd.read_csv("data.csv")
rsum = robjects.r['experiment']

NR_LINES_IN_SAMPLE = _printing_line_count(data)
_NR_PROPOSED_LINES = 8 # don't change this




In [10]:
if len(data['resistance']) == NR_LINES_IN_SAMPLE:
    # print(len(d['resistance']))
    print(f"First batch proposal with {NR_LINES_IN_SAMPLE} lines to train")
    rsum((1))
    duplicate_to_dataset(nr_proposed_lines=_NR_PROPOSED_LINES, prepattern=False)

elif len(data['resistance']) < NR_LINES_IN_SAMPLE:
    print ("Not enough data points in dataset. Exiting.")
    exit()

else:
    print(f"Further batch proposals of {NR_LINES_IN_SAMPLE} lines")
    proposed = data['resistance'].iloc[-NR_LINES_IN_SAMPLE:]
    rsum((list(proposed)))
    duplicate_to_dataset(nr_proposed_lines=_NR_PROPOSED_LINES)

Further batch proposals of 53 lines


R[write to console]: Loading required package: mlr

R[write to console]: Loading required package: ParamHelpers

R[write to console]: Warning message: 'mlr' is in 'maintenance-only' mode since July 2019.
Future development will only happen in 'mlr3'
(<https://mlr3.mlr-org.com>). Due to the focus on 'mlr3' there might be
uncaught bugs meanwhile in {mlr} - please consider switching.

R[write to console]: Loading required package: smoof

R[write to console]: Loading required package: checkmate

R[write to console]: Loading required package: R.oo

R[write to console]: Loading required package: R.methodsS3

R[write to console]: R.methodsS3 v1.8.1 (2020-08-26 16:20:06 UTC) successfully loaded. See ?R.methodsS3 for help.

R[write to console]: R.oo v1.24.0 (2020-08-26 16:11:58 UTC) successfully loaded. See ?R.oo for help.

R[write to console]: 
Attaching package: 'R.oo'


R[write to console]: The following object is masked from 'package:R.methodsS3':

    throw


R[write to console]: The follo

        se      mean
1 71619022   9262391
2 79181176  13272640
3 70456664  21681112
4 78705427  16800882
5 77023755  11650759
6 45205074 -16983053
7 76668758  13076858
8 76012864  12977432
     power time pressure
278    964 3971       75
356    714 2598       75
1189  2408 5063        1
1370   695 3067       74
8058   841 2737       75
89    2600 5513        1
112    627 2772       75
102    664 3147       75
[1] "Proposed parameters:"


R[write to console]: Error in `[.data.frame`(read.csv("data.csv", header = FALSE, skip = nL -  : 
  undefined columns selected

R[write to console]: In addition: 
R[write to console]: Warning messages:

R[write to console]: 1: 
R[write to console]: In if (s == 1) { :
R[write to console]: 
 
R[write to console]:  the condition has length > 1 and only the first element will be used

R[write to console]: 2: package 'R.utils' was built under R version 4.1.3 

R[write to console]: 3: package 'R.oo' was built under R version 4.1.1 

R[write to console]: 4: package 'R.methodsS3' was built under R version 4.1.1 



RRuntimeError: Error in `[.data.frame`(read.csv("data.csv", header = FALSE, skip = nL -  : 
  undefined columns selected


In [11]:
data.loc[NR_LINES_IN_SAMPLE:NR_LINES_IN_SAMPLE+8]


,power,time,pressure,ratio,resistance
53,964,3971,800,NaN,NaN
54,714,2598,800,NaN,NaN
55,2408,5063,60,NaN,NaN
56,695,3067,790,NaN,NaN
57,841,2737,800,NaN,NaN
58,2600,5513,60,NaN,NaN
59,627,2772,800,NaN,NaN
60,664,3147,800,NaN,NaN


# (M) Begin the experiment

In [12]:


measurement_times = []

print(f'Here is the last group and next group of data points:\n{data}')

def batch_experiment(total_lines) -> None:

    print("Loading data from csv")
    data:pd.DataFrame = pd.read_csv('data.csv')
    print("Batch Experiment is starting")
    _mirror_move_time:float = .5 # in seconds
    _line_length:float = 2 # in mm
    vacuum_time:float = 2 # in minutes
    line_coordinates:list = generate_line_coordinates(total_lines)
    line_parameters:list = get_line_parameters(data, total_lines)
    line_numbers:list = get_line_numbers(data, total_lines)

    _spot_measured = 9
    TARGET:str = 'GD'
    print(f"Here's the number of total lines:{total_lines}")
    if (len(line_coordinates) != len(line_parameters)) | (len(line_coordinates) != len(line_numbers)):
        print('Print variables do no match')
        exit()

    for i in range(total_lines):
        line_start_time = time.time()
        laser_power.power = 0.01
        save_line:int = line_numbers[i]
        parameters:pd.core.series.Series = line_parameters.loc[i]

        print(f"====== Printing line #{i+1}/{total_lines}")
        print(f"Saving to line {save_line}")

        print(f"\nCurrent patterning parameters are:\n {parameters}")
        time_of_file:float = parameters['time']
        power_of_file:float = parameters['power']
        pressure_of_file:float = parameters['pressure']


        line_i_coorinates:tuple = line_coordinates[i]
        line_start_x_coordinate:float = line_i_coorinates[0]
        line_i_y_coordinate:float = line_i_coorinates[1]
        xps.move_stage('XYZ.X',line_start_x_coordinate)
        xps.move_stage('XYZ.Y',line_i_y_coordinate)
        pos_all()
        print("\n\n")
        data:pd.DataFrame = pd.read_csv('data.csv')
        plot_data_df:pd.DataFrame = pd.read_csv('plot_data.csv')

        print("Vacuum-infill in progress...")
     
        to_ambient()
        print(f"1st ambient: {current_pressure()}")
        time.sleep(1)
        to_ambient()
        print(f"2nd ambient: {current_pressure()}")
        time.sleep(1)
        to_ambient()
        print(f"3rd ambient: {current_pressure()}")
        time.sleep(5)
        laser_power.write("OPMODE=Off")
        
        to_vacuum(vacuum_time)
        time.sleep(5)
        print(f"vac : {current_pressure()}")

        laser_power.power = 0.01

        gopr(pressure_of_file)
        
        # for j in range(3):
        #     # print(f"Pressure count {j}")
        #     gopr(pressure_of_file)
        #     close_all()
        #     time.sleep(5)

        laser_power.write("OPMODE=On")
        print(f"\n Pressure of file is :{pressure_of_file}\n")
        print(f"\n Pressure is now :{current_pressure()}\n")

        xps.define_line_trajectories(start=float(line_start_x_coordinate),
                                 stop=float(line_start_x_coordinate)+_line_length,
                                 step=0.01,
                                 scantime=(time_of_file/1000),
                                 axis='X')

        xps.download_trajectory('foreward.trj')

        d = pd.read_csv('foreward.trj', header=None)
        d = d.loc[:, (d != 0).any(axis=0)]
        d.columns = ['ramptime','rampdist','rampvel']
        print(d)

        total_time = np.sum(d['ramptime'])
        mirror_sleep = d['ramptime'][0] - _mirror_move_time
        print(f'Total time: {total_time}, Mirror_sleep: {mirror_sleep} in MAIN')

        # PATTERN EXPERIMENT
        print("Power is now: {} mW".format(power_of_file))
        xps.move_stage('XYZ.X',(float(line_start_x_coordinate) - d['rampdist'][0]))

        pos_all()
        time.sleep(15)
        print("\n\n")
        mirror("off")
        pp=power_of_file
        laser_power.power = (pp/0.2163)/1000
        print("Allowing 60 seconds for laser to ramp")
        time.sleep(60)
        
        
        print("###### Starting pattern in 5 seconds! #########")
        ##Draw Lines
        time.sleep(5)
        
        import sys
        motion_thread = MotionThread()
        mirror_thread = MirrorThread()

        motion_thread.start()
        mirror_thread.start()

        motion_thread.join()
        mirror_thread.join()

        time.sleep(10)
        print("\n Job done\n")

        ##raman spectra analysis
        laser_power.power=0.1
        time.sleep(15)

        line_stop_time = time.time()
        total_time = line_stop_time - line_start_time
        print(f"Line took: {total_time}")
        measurement_times.append(total_time)
        print(f"Measurement Times = {measurement_times}")
    ########################################

    # post patterning raman analysis

        check=[float(line_start_x_coordinate) + .2*_line_length, \
                float(line_start_x_coordinate) + .4*_line_length, \
                float(line_start_x_coordinate) + .6*_line_length]
        # mm=0
        for spot in range(3):
            print(f"############### SPOT {spot} ##################")
            xps.move_stage('XYZ.X',check[spot])
            print(pos_all())

            for iii in range(3):
                print(f"############# iteration {iii} ###############")
                mirror("on")
                #writing G/D of 3 spots in 3 lines inside dataset.csv
        #         capture_photo("on",spot,line)
                print(f"Measuring patterned line for {TARGET}")
                GD = capture_photo("on", spot, save_line, iii, target=TARGET)

                mirror("off")
                time.sleep(5)

        # if save_line <= TOTAL_LINES:
        # print(f"############# LINE {save_line} <= {TOTAL_LINES}, CURRENT_STEP={steps}, TAKING MEAN: #############")
        # get_mean(steps, save_line, _spot_measured)




Here is the last group and next group of data points:
    power   time  pressure  ratio  resistance
0     701  12183       540    0.0   50.748620
1     914  10647       120    0.0   71.062688
2    1635   8049       220    0.0   47.701587
3     480   9892       250    0.0   56.782144
4     342  11576       390    0.0  118.670146
..    ...    ...       ...    ...         ...
56    695   3067       790    NaN         NaN
57    841   2737       800    NaN         NaN
58   2600   5513        60    NaN         NaN
59    627   2772       800    NaN         NaN
60    664   3147       800    NaN         NaN

[61 rows x 5 columns]


In [13]:
intensity_check =capture_photo('adjust',0,0,0)
print("---Final Verification of Batch Experiment---")
print("Current Pressure: ", current_pressure())
pos_all()
print(f'Here is the intensity:{intensity_check}')
print("-----------------------------------------")
print("-----------------------------------------")

try:
        laser_power.write("OPMODE=On")
        batch_experiment(total_lines=8)
except:
        to_ambient()
        laser_power.write("OPMODE=Off")
        sys.exit()

laser_power.write("OPMODE=Off")

MIRROR UP
 
MIRROR DOWN
---Final Verification of Batch Experiment---
Current Pressure:  121
XYZ.X: 4.5000016524
XYZ.Y: 11.9999985066
XYZ.Z: 3.4900030908
Here is the intensity:132.33333333333334
-----------------------------------------
-----------------------------------------
Loading data from csv
Batch Experiment is starting
Here's the number of total lines:8
====== Printing line #1/8
Saving to line 53

Current patterning parameters are:
 index           53.0
power          964.0
time          3971.0
pressure       800.0
ratio            NaN
resistance       NaN
Name: 0, dtype: float64
XYZ.X: 1.5000064506
XYZ.Y: 6.500193147
XYZ.Z: 3.4900030908



Vacuum-infill in progress...
1st ambient: 32
2nd ambient: 16
3rd ambient: 14
vac : 1
done

 Pressure of file is :800.0


 Pressure is now :799

   ramptime  rampdist   rampvel
0  0.302191  0.152199  0.503651
1  3.990855  2.010000  0.503651
2  0.302191  0.152199  0.000000
Total time: 4.595236999999999, Mirror_sleep: -0.197809 in MAIN
Power is

In [ ]:
to_ambient()
to_vacuum(0.01)
laser_power.write("OPMODE=Off")
xps.kill_group('XYZ')
xps.initialize_allgroups()
xps.home_allgroups()

XPSError: message= homing group 'XYZ', error=-26, description=Kill command or Emergency signal : check each positioners and each slave positioners, check that motion does not exceed software limits when combined with mapping and other features.


XPSException: homing group 'XYZ' Kill command or Emergency signal : check each positioners and each slave positioners, check that motion does not exceed software limits when combined with mapping and other features. [Error -26]

In [ ]:
print(measurement_times)

[1213.434579372406, 285.94849920272827, 356.89261054992676, 1582.502274274826, 340.95546317100525, 286.9056932926178, 536.9851233959198, 443.5903744697571]
